this notebook creates two maps that show the Hunza watershed with ESRI imagery
- first map overlays the algorithm calculated melt onset dates (MOD) for SIR pixels
- second map overlays the algorithm calculated melt onset dates (MOD) for GRD pixels

the pixel overlays are simply a scatter plot with rectangular markers and are not using EASE-grid projection

## Load in all the modules needed

In [ ]:
%pylab notebook
import matplotlib.pyplot as plt
from netCDF4 import Dataset, num2date
import numpy as np
import pandas as pd
from cetbtools.ease2conv import Ease2Transform
from mpl_toolkits.basemap import Basemap
import time

In [ ]:
# navigate to where scripts are saved
%cd /projects/brodzik/ipython_notebooks/melt_onset/scripts/
%ls

In [ ]:
# load the custom functions
from CETB_read_functions import read_Tb
from CETB_read_functions import coords
from CETB_read_functions import calc_DAV
from CETB_read_functions import find_UIB_cube_offset
from CETB_read_functions import grid_locations_of_UIB
from CETB_algorithms import DAV_MOD
from CETB_analysis import MOD_array
from CETB_analysis import MOD_array_year

## Specify inputs

In [ ]:
#Specify region, satellite, sensor, channel, and image reconstruction algorithm of interest in file name
# this notebook will read in 2 CETB datasets so that channels/algorithms/sensors can be compared
region='UIB'  #make this the same syntax as cubefilenames and sub-directory
platform='AQUA'   #'AQUA' for AMSRE, 'F13','F14','F15'... for SSMI
sensor='AMSRE'  #'AMSRE', 'SSMI', etc.
channel='36V'  #'36V','36H', '18V','18H', etc. '19V','19H' and '37V','37H' for SSMI)
version='v1.3'
proj='N'

if sensor=='SSMI':
    provider='CSU' 
elif sensor=='AMSRE':
    provider='RSS'

cubeDir = '/work/PMESDR/CETB_v1.3/%s_%s/%s/cubes_%s/' % (platform, sensor, proj, region)    

# prefix filepath
prefix_GRD = 'CETB.cubefile.%s.%s_%s-%s-GRD-%s-%s' % (region, platform, sensor, channel, provider, version)
prefix_SIR = 'CETB.cubefile.%s.%s_%s-%s-SIR-%s-%s' % (region, platform, sensor, channel, provider, version)

# years for each sensor
if platform=='F13':
    Years = [2002,2003,2004,2005,2006,2007,2008,2009]
elif platform=='F14':
    Years=[2002,2003,2004,2005,2006,2007,2008]
elif platform=='F15':
    Years=[2002,2003,2004,2005,2006,2007,2008,2009,2010,2011]
elif platform=='AQUA':
    Years=[2003,2004,2005,2006,2007,2008,2009,2010,2011]

#Enter a site name for titles of plots
Site='Hunza'

In [ ]:
# SPECIFY latitude and longitude in decimal degrees, need to choose lat/lon corners so that we will load
# in a rectangle of pixels within the corners of these coordinates
lat_start=35.9
lat_end=37.1
lon_start=74
lon_end=76

In [ ]:
# get cube offset for finding row/col
# function is region specific
find_UIB_cube_offset(cubeDir, cubeType=None, verbose=True)

In [ ]:
# get the GRD pixel IDs for the lat/lon rectangle chosen
rows_cols_GRD=coords(cubeDir, prefix_GRD, lat_start, lat_end, lon_start, lon_end)
rows_cols_GRD

In [ ]:
# row and col numbers of all the SIR pixels contained by the GRD pixels
# FIXME: this assumes SIR is 3.125 km data
rows_cols_env=[rows_cols_GRD[0]*8,rows_cols_GRD[1]*8, rows_cols_GRD[2]*8, rows_cols_GRD[3]*8]
rows_cols_env

In [ ]:
# load in SIR TB data
data_SIR=read_Tb(cubeDir, prefix_SIR, Years,rows_cols_env[0],rows_cols_env[1],rows_cols_env[2],rows_cols_env[3])
CETB_SIR=data_SIR[0]   # 3-D Tb time-series array of TB
cal_date=data_SIR[1]    # 1-D array of dates, these will get passed to later functions
cal_year=data_SIR[2]    # 1-D array of years
cal_month=data_SIR[3]   # 1-D array of months

# load GRD Tb data
data_GRD=read_Tb(cubeDir, prefix_GRD, Years,rows_cols_GRD[0],rows_cols_GRD[1],rows_cols_GRD[2],rows_cols_GRD[3])
CETB_GRD=data_GRD[0]

In [ ]:
#CETB_SIR.shape, CETB_SIR.dtype, np.nanmin(CETB_SIR), np.nanmax(CETB_SIR)

In [ ]:
# calculate DAV for the Tb data that was imported
DAV_SIR=calc_DAV(CETB_SIR)
DAV_GRD=calc_DAV(CETB_GRD)

## get the latitudes and longitudes of each pixel for plotting

In [ ]:
#Get lat/lon of each pixel in the subset of 64
# these get used for plotting over basemap
filename="%s%s.%4d.TB.nc" % (cubeDir, prefix_SIR, 2003)
data=Dataset(filename, "r", format="NETCDF4")

lat=data.variables['latitude'] 
lat=lat[:]
lon=data.variables['longitude']
lon=lon[:]
lat_lon=np.dstack((lat,lon))

coordinates=lat_lon[rows_cols_env[0]:rows_cols_env[1],rows_cols_env[2]:rows_cols_env[3]]

coordinates=coordinates.reshape(((rows_cols_env[1]-rows_cols_env[0])*(rows_cols_env[3]-rows_cols_env[2])),2)
pixel_lats=coordinates[:,0]
pixel_lons=coordinates[:,1]

pixel_lats

In [ ]:
#Get lat/lon for GRD pixels
# similar to SIR above
filename="%s%s.%4d.TB.nc" % (cubeDir, prefix_GRD, 2003)
data=Dataset(filename, "r", format="NETCDF4")

lat=data.variables['latitude'] 
lat=lat[:]
lon=data.variables['longitude']
lon=lon[:]
lat_lon=np.dstack((lat,lon))

coordinates=lat_lon[rows_cols_GRD[0]:rows_cols_GRD[1],rows_cols_GRD[2]:rows_cols_GRD[3]]

coordinates=coordinates.reshape(((rows_cols_GRD[1]-rows_cols_GRD[0])*(rows_cols_GRD[3]-rows_cols_GRD[2])),2)
pixel_lats_GRD=coordinates[:,0]
pixel_lons_GRD=coordinates[:,1]

pixel_lats_GRD

## create arrays of MOD to be used for plotting

In [ ]:
# get array of average MODs for SIR pixels for all the years loaded
window=10   # window for MOD algorithm, '10' would be 5 days (2 measurements per day)
count=3    # number of Tb/DAV exceedances to trigger MOD
DAV_threshold=40
Tb_threshold=252
year=2003

In [ ]:
Years

In [ ]:
# sir MOD array - MOD will be in day of year (DOY)
MOD_DOY_array=MOD_array(cubeDir, prefix_SIR, CETB_SIR, DAV_SIR, 
                        rows_cols_env, cal_date, Years, window, count, DAV_threshold, Tb_threshold)
MOD_DOY_array

In [ ]:
# get array of MOD for each pixel SIR - one year of interest
#window=10
#count=3
#DAV_threshold=18
#Tb_threshold=252
#year=2003

t = time.process_time()
MOD_DOY_array_year=MOD_array_year(cubeDir, prefix_SIR, CETB_SIR, DAV_SIR, 
                                  rows_cols_env, cal_date, year, window, count, DAV_threshold, Tb_threshold)
elapsed_time = time.process_time() - t
MOD_DOY_array_year, elapsed_time

In [ ]:
# MOD of the GRD pixel - avg all years
#window=10
#count=3
#DAV_threshold=18
#Tb_threshold=252

MOD_DOY_array_GRD=MOD_array(cubeDir, prefix_GRD, CETB_GRD, DAV_GRD, 
                            rows_cols_GRD, cal_date, Years, window, count, DAV_threshold, Tb_threshold)
MOD_DOY_array_GRD

In [ ]:
# get MOD for GRD pixel - one year of interest
#window=10
#count=3
#DAV_threshold=18
#Tb_threshold=252
#year=2003

t = time.process_time()

MOD_DOY_array_GRD_year=MOD_array_year(cubeDir, prefix_GRD, CETB_GRD, DAV_GRD, 
                                      rows_cols_GRD, cal_date, year, window, count, DAV_threshold, Tb_threshold)
elapsed_time = time.process_time() - t
MOD_DOY_array_GRD_year, elapsed_time

# Save the original data as produced by Mitch's version

In [ ]:
# Save these arrays to numpy zip
outfile = '/projects/brodzik/pmesdr_melt_onset/data/Hunza_MOD.npz'
#np.savez(outfile, 
#         MOD_DOY_array=MOD_DOY_array, 
#         MOD_DOY_array_year=MOD_DOY_array_year, 
#         MOD_DOY_array_GRD=MOD_DOY_array_GRD, 
#         MOD_DOY_array_GRD_year=MOD_DOY_array_GRD_year)
npzfile = np.load(outfile)
npzfile.files
# should return 4 variable names as a dict
orig_MOD_DOY_array = npzfile['MOD_DOY_array']
orig_MOD_DOY_array_year = npzfile['MOD_DOY_array_year']
orig_MOD_DOY_array_GRD = npzfile['MOD_DOY_array_GRD']
orig_MOD_DOY_array_GRD_year = npzfile['MOD_DOY_array_GRD_year']


# Save the new data after my mods to make things fast 
# and removing the date shift of one day

In [ ]:
# Save these arrays to numpy zip
outfile = '/projects/brodzik/pmesdr_melt_onset/data/Hunza_MOD.fast_no_date_shift.npz'
np.savez(outfile, 
         MOD_DOY_array=MOD_DOY_array, 
         MOD_DOY_array_year=MOD_DOY_array_year, 
         MOD_DOY_array_GRD=MOD_DOY_array_GRD, 
         MOD_DOY_array_GRD_year=MOD_DOY_array_GRD_year)
#npzfile = np.load(outfile)
#npzfile.files
# should return 4 variable names as a dict
#MOD_DOY_array = npzfile['MOD_DOY_array']
#MOD_DOY_array_year = npzfile['MOD_DOY_array_year']
#MOD_DOY_array_GRD = npzfile['MOD_DOY_array_GRD']
#MOD_DOY_array_GRD_year = npzfile['MOD_DOY_array_GRD_year']


In [ ]:
orig_MOD_DOY_array, MOD_DOY_array

In [ ]:
diff = MOD_DOY_array - orig_MOD_DOY_array
print(np.nanmin(diff), np.nanmax(diff))

In [ ]:
orig_MOD_DOY_array_year, MOD_DOY_array_year

In [ ]:
diff = MOD_DOY_array_year - orig_MOD_DOY_array_year
print(np.nanmin(diff), np.nanmax(diff))

In [ ]:
orig_MOD_DOY_array_GRD, MOD_DOY_array_GRD

In [ ]:
diff = MOD_DOY_array_GRD - orig_MOD_DOY_array_GRD
print(np.nanmin(diff), np.nanmax(diff))

In [ ]:
orig_MOD_DOY_array_GRD_year, MOD_DOY_array_GRD_year

In [ ]:
diff = MOD_DOY_array_GRD_year - orig_MOD_DOY_array_GRD_year
print(np.nanmin(diff), np.nanmax(diff))

In [ ]:
# Convert the MOD information back to an image for display
MOD_DOY_array_GRD.shape, rows_cols_GRD

In [ ]:
MOD_DOY = MOD_DOY_array.reshape((rows_cols_env[1] - rows_cols_env[0],
                                 rows_cols_env[3] - rows_cols_env[2]))
MOD_DOY_year = MOD_DOY_array_year.reshape((rows_cols_env[1] - rows_cols_env[0],
                                           rows_cols_env[3] - rows_cols_env[2]))
                                         
MOD_DOY_GRD = MOD_DOY_array_GRD.reshape((rows_cols_GRD[1] - rows_cols_GRD[0],
                                         rows_cols_GRD[3] - rows_cols_GRD[2]))
MOD_DOY_GRD_year = MOD_DOY_array_GRD_year.reshape((rows_cols_GRD[1] - rows_cols_GRD[0],
                                                   rows_cols_GRD[3] - rows_cols_GRD[2]))

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(10,6))

label = "SIR Avg MOD (%4d-%4d)" % (Years[0], Years[-1])
im0 = ax[0,0].imshow(np.rot90(MOD_DOY, -1), cmap='RdBu', 
                    interpolation='None', vmin=np.min(75), vmax=np.max(170), label=label)
cbar0 = fig.colorbar(im0, ax=ax[0,0])
cbar0.ax.set_ylabel('DOY')
ax[0,0].axis('off')
ax[0,0].set_title(label)

label = "SIR MOD (%s)" % year
im1 = ax[0,1].imshow(np.rot90(MOD_DOY_year, -1), cmap='RdBu', 
                    interpolation='None', vmin=np.min(75), vmax=np.max(170), label=label)
fig.colorbar(im1, ax=ax[0,1])
ax[0,1].axis('off')
ax[0,1].set_title(label)

label = "GRD Avg MOD (%4d-%4d)" % (Years[0], Years[-1])
im2 = ax[1,0].imshow(np.rot90(MOD_DOY_GRD, -1), cmap='RdBu', 
                    interpolation='None', vmin=np.min(75), vmax=np.max(170), label=label)
fig.colorbar(im2, ax=ax[1,0])
ax[1,0].axis('off')
ax[1,0].set_title(label)

label = "GRD MOD (%s)" % year
im3 = ax[1,1].imshow(np.rot90(MOD_DOY_GRD_year, -1), cmap='RdBu', 
                    interpolation='None', vmin=np.min(75), vmax=np.max(170), label=label)
fig.colorbar(im3, ax=ax[1,1])
ax[1,1].axis('off')
ax[1,1].set_title(label)

plt.tight_layout()
#fig.savefig("./test.SIR-avg-std-min-max.t%03d.png" % (bandnum-1), dpi=300)

## create the maps

In [ ]:
# PLOT the MOD MAP - avg of all years or one year of interest - SIR

# Choose plot type - 'year' for one year of interest, 'all' for average of all years
#plot_type='year'  
plot_type='all'

#year=2003


if plot_type=='all':
    array=MOD_DOY_array
    title = "%s-%s - MOD - (Avg DOY %4d-%4d)" % (platform, Site, Years[0], Years[-1])
elif plot_type=='year':
    array=MOD_DOY_array_year
    title = "%s-%s - MOD - (%4d DOY)" % (platform, Site, year)

fig = plt.figure(figsize=(8,8))
# create figure and axes instances
#fig = plt.figure()
ax = fig.add_axes([0.1,0.1,0.8,0.8])
#ax = fig.add_subplot(121)
# create polar stereographic Basemap instance.
# Why polar stereo???
m = Basemap(projection='stere',lon_0=(lon_start+lon_end)/2,lat_0=(lat_start+lat_end)/2,lat_ts=90.,\
            llcrnrlat=(lat_start-.3),urcrnrlat=(lat_end+.5),\
            llcrnrlon=(lon_start-.3),urcrnrlon=(lon_end+.3),\
            rsphere=6371200.,resolution='l',area_thresh=10000, epsg=3857)
m.drawcoastlines()
m.drawcountries()
m.drawstates()
#m.etopo(scale=5, alpha=0.7)
#m.bluemarble(scale=1)
parallels = np.arange(0.,90,0.5)
m.drawparallels(parallels,labels=[1,0,0,0],fontsize=10)
# draw meridians
meridians = np.arange(0,360.,0.5)
m.drawmeridians(meridians,labels=[0,0,0,1],fontsize=10)
m.arcgisimage(service='World_Physical_Map', xpixels = 1500, verbose= True)

m.readshapefile('/work/charis/ti_model/basins/basin_outlines/IN_Hunza_at_DainyorBridge', 'Hunza', color='red', linewidth=1)

x,y=m(pixel_lons, pixel_lats)
m.scatter(x,y,c=array, s=30, marker='s',lw=0,cmap='RdBu', alpha=.6)
plt.title(title)

plt.colorbar(label='DOY')
plt.clim(75,175)  #color bar limits

plt.show()


# plot another blank map with no pixel overlay
fig = plt.figure(figsize=(8,8))
ax = fig.add_axes([0.1,0.1,0.8,0.8])

# create polar stereographic Basemap instance.
m = Basemap(projection='stere',lon_0=(lon_start+lon_end)/2,lat_0=(lat_start+lat_end)/2,lat_ts=90.,\
            llcrnrlat=(lat_start-.3),urcrnrlat=(lat_end+.5),\
            llcrnrlon=(lon_start-.3),urcrnrlon=(lon_end+.3),\
            rsphere=6371200.,resolution='l',area_thresh=10000, epsg=3857)
m.drawcoastlines()
m.drawcountries()
m.drawstates()
#m.etopo(scale=5, alpha=0.7)
#m.bluemarble(scale=1)
parallels = np.arange(0.,90,0.5)
m.drawparallels(parallels,labels=[1,0,0,0],fontsize=10)
# draw meridians
meridians = np.arange(180.,360.,0.5)
m.drawmeridians(meridians,labels=[0,0,0,1],fontsize=10)
m.arcgisimage(service='World_Physical_Map', xpixels = 1500, verbose= True)


plt.title(title)
plt.show()


In [ ]:
# PLOT the MOD MAP - avg of all years or one year of interest - GRD
# Choose plot type - 'year' for one year of interest, 'all' for average of all years
#plot_type='year'  
plot_type='all'

#year=2003


if plot_type=='all':
    array=MOD_DOY_array_GRD
    title = "%s-%s - MOD - (Avg DOY %4d-%4d)" % (platform, Site, Years[0], Years[-1])
elif plot_type=='year':
    array=MOD_DOY_array_GRD_year
    title = "%s-%s - MOD - (%4d DOY)" % (platform, Site, year)

fig = plt.figure(figsize=(8,8))
# create figure and axes instances
#fig = plt.figure()
ax = fig.add_axes([0.1,0.1,0.8,0.8])
#ax = fig.add_subplot(121)
# create polar stereographic Basemap instance.
m = Basemap(projection='stere',lon_0=(lon_start+lon_end)/2,lat_0=(lat_start+lat_end)/2,lat_ts=90.,\
            llcrnrlat=(lat_start-.3),urcrnrlat=(lat_end+.5),\
            llcrnrlon=(lon_start-.3),urcrnrlon=(lon_end+.3),\
            rsphere=6371200.,resolution='l',area_thresh=10000, epsg=3857)
m.drawcoastlines()
m.drawcountries()
#m.drawstates()
#m.etopo(scale=3, alpha=0.7)
#m.bluemarble(scale=1)
parallels = np.arange(0.,90,0.5)
m.drawparallels(parallels,labels=[1,0,0,0],fontsize=10)
# draw meridians
meridians = np.arange(0,360.,0.5)
m.drawmeridians(meridians,labels=[0,0,0,1],fontsize=10)
m.arcgisimage(service='World_Physical_Map', xpixels = 1500, verbose= True)

m.readshapefile('/work/charis/ti_model/basins/basin_outlines/IN_Hunza_at_DainyorBridge', 
                'Hunza', color='red', linewidth=1)

x,y=m(pixel_lons_GRD, pixel_lats_GRD)
m.scatter(x,y,c=array, s=1200, marker='s',lw=0,cmap='RdBu', alpha=.6)

plt.title(title)

plt.colorbar(label='DOY')
plt.clim(75,175)  #color bar limits

plt.show()


# plot another blank map with no pixel overlay
fig = plt.figure(figsize=(8,8))
ax = fig.add_axes([0.1,0.1,0.8,0.8])

# create polar stereographic Basemap instance.
m = Basemap(projection='stere',lon_0=(lon_start+lon_end)/2,lat_0=(lat_start+lat_end)/2,lat_ts=90.,\
            llcrnrlat=(lat_start-.3),urcrnrlat=(lat_end+.5),\
            llcrnrlon=(lon_start-.3),urcrnrlon=(lon_end+.3),\
            rsphere=6371200.,resolution='l',area_thresh=10000, epsg=3857)
m.drawcoastlines()
m.drawcountries()
m.drawstates()
#m.etopo(scale=5, alpha=0.7)
#m.bluemarble(scale=1)
parallels = np.arange(0.,90,0.5)
m.drawparallels(parallels,labels=[1,0,0,0],fontsize=10)
# draw meridians
meridians = np.arange(180.,360.,0.5)
m.drawmeridians(meridians,labels=[0,0,0,1],fontsize=10)
m.arcgisimage(service='World_Physical_Map', xpixels = 1500, verbose= True)
m.readshapefile('/work/charis/ti_model/basins/basin_outlines/IN_Hunza_at_DainyorBridge', 
                'Hunza', color='red', linewidth=1)

plt.title(title)
plt.show()